In [ ]:
#normalizar os arquivos de teste

import pandas as pd
import os

raiz = "large_scale"

meta = "large_scale_trated"


all_files = sorted([os.path.join(root, f) for root, _, files in os.walk(raiz) for f in files])
for i in range(0, len(all_files),2):
    info_file = all_files[i]
    items_file = all_files[i+1]

    info_df = pd.read_csv(info_file, header=None)
    for index, row in info_df.iterrows():
        if row[0] == 'n':
            n = int(row[1])
        elif row[0] == 'c':
            wmax = int(row[1])

    # Processa o arquivo items correspondente
    items_df = pd.read_csv(items_file)[[' price', ' weight']]
        
    # Adiciona n e wmax como primeira linha
    modified_df = pd.concat([
        pd.DataFrame([[n, wmax]], columns=items_df.columns),
        items_df
    ], ignore_index=True)

    base_name = os.path.basename(items_file).split('.')[0]  # Remove extensão
    output_file = os.path.join(meta, f"{base_name}.txt")
        
        # Salva como TXT com espaços
    with open(output_file, 'w') as f:
        for _, row in modified_df.iterrows():
            f.write(f"{int(row[0])} {int(row[1])}\n")
    
    print(f"✓ Arquivo gerado: {os.path.basename(output_file)}")
    



In [18]:
import pandas as pd
import re

def parse_file(path, name):
    data = []
    current_test = {}  # Armazenará os dados completos de um teste (BD + Aprox)
    last_method = None
    
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            
            # Parse Número de Total de Itens e Peso máximo
            if line.startswith('Número de Total de Itens:'):
                match = re.match(r'Número de Total de Itens: (\d+) Peso máximo: (\d+)', line)
                if match:
                    # Se já temos dados do método anterior, prepara para novo teste
                    if last_method == '2-aproximativo':
                        data.append(current_test)
                        current_test = {}
                    
                    current_test['n'] = match.group(1)
                    current_test['wMax'] = match.group(2)
            
            # Parse Branch and Bound value
            elif line.startswith('Branch and Bound:'):
                current_test['Valor BD'] = line.split(': ')[1].strip()
                last_method = 'BD'
            
            # Parse 2-aproximativo value
            elif line.startswith('2-aproximativo:'):
                current_test['Valor 2-Aprox'] = line.split(': ')[1].strip()
                last_method = '2-aproximativo'
            
            # Parse Branch and Bound time
            elif line.startswith('Tempo Branch and Bound :'):
                current_test['Tempo BD'] = line.split(': ')[1].strip()
            
            # Parse Aproximativo time
            elif line.startswith('Tempo Aproximativo :'):
                current_test['Tempo 2-Aprox'] = line.split(': ')[1].strip()
            
            # Parse RSS memory for BD
            elif line.startswith('* Memória residente real (RSS):') and '[BD]' in current_test.get('last_section', ''):
                current_test['Custo BD'] = line.split(': ')[1].replace(' KB', '').strip()
            
            # Parse RSS memory for aprox
            elif line.startswith('* Memória residente real (RSS):') and '[aprox]' in current_test.get('last_section', ''):
                current_test['Custo 2-Aprox'] = line.split(': ')[1].replace(' KB', '').strip()
                current_test['Tipo'] = name
            # Track current section
            elif line.startswith('=== Métricas de Memória'):
                current_test['last_section'] = line
    
    # Add the last test if exists
    if current_test:
        data.append(current_test)
    
    return pd.DataFrame(data)

# Process both files
df_large = parse_file("large_results.txt", "Large")
df_low = parse_file("low_results.txt", "Low")

# Combine both DataFrames
df_combined = pd.concat([df_large, df_low], ignore_index=True)

# Convert scientific notation to normal numbers and clean data
def clean_value(val):
    if not isinstance(val, str):
        return val
    if 'e+' in val:
        return str(float(val))
    return val.replace(' KB', '')

df_combined = df_combined.applymap(clean_value)

# Reorder columns
columns = ['Tipo', 'n', 'wMax', 'Valor BD', 'Tempo BD', 'Custo BD', 
           'Valor 2-Aprox', 'Tempo 2-Aprox', 'Custo 2-Aprox']
df_combined = df_combined[columns]

# Fill empty values with empty string for better CSV output
df_combined = df_combined.fillna('')

# Save to CSV
df_combined.to_csv('resultados_combinados.csv', index=False)

print("DataFrame consolidado:")
print(df_combined)

DataFrame consolidado:
     Tipo    n   wMax Valor BD Tempo BD Custo BD Valor 2-Aprox Tempo 2-Aprox  \
0   Large  200    997     5397     2470     1876          5397        522695   
1   Large  500   2543    28857   552309     1916         28857     1131140.0   
2   Large  500   2543     4566    85657     1896          4566        673646   
3   Large  500   2517     7117   491068     1936          7117     1200350.0   
4     Low   20    879     1025     4231     1900          1025           544   
5     Low   10    269      295       33     1928           293            68   
6     Low   20    878     1024     4192     1932          1024           523   
7     Low    4     20       35       27     1940            35            31   
8     Low    4     11       23       19     1920            23            23   
9     Low   15    375  481.069       98     1932       481.069           236   
10    Low   10     60       52      190     1988            52           130   
11    Low    7   

/tmp/ipykernel_55923/990892569.py:76: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_combined = df_combined.applymap(clean_value)
